# Notebook 1: Data Fetching
This notebook fetches historical stock data for selected companies from Yahoo Finance using the `yfinance` library. The data is stored in an SQLite database for further processing.

## Importing Necessary Libraries
The libraries used in this notebook include:
- `yfinance`: For fetching stock data.
- `sqlite3`: For interacting with the SQLite database.
- `pandas`: For data manipulation and storage.

In [1]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import sqlite3

## Fetching Data from Yahoo Finance
The `fetch_and_store_stock_data` function:
- Retrieves data for the given ticker using the `yfinance` library.
- Adds metadata such as `Company` and `Ticker` columns.
- Saves the data into the SQLite database in the table `stocks`.
Error handling ensures the process skips companies that encounter issues during data retrieval.


In [2]:
# Define a function to fetch stock data and save to SQLite
def fetch_and_store_data(ticker, company_name, db_path):
    """
    Fetch stock data for a specific ticker and save it to an SQLite database.
    Parameters:
    ticker (str): The stock ticker symbol (e.g., 'XOM').
    company_name (str): The full company name.
    db_path (str): Path to the SQLite database file.
    """
    try:
        # Fetch data from yfinance
        stock_data = yf.download(ticker, start='2000-01-01', end='2024-12-31', progress=True)
        
        # Flatten multi-level column headers (if any)
        stock_data.columns = [col[0] if isinstance(col, tuple) else col for col in stock_data.columns]
        
        # Add additional columns for the company and ticker
        stock_data['Company'] = company_name
        stock_data['Ticker'] = ticker
        
        # Reset the index to make 'Date' a column
        stock_data.reset_index(inplace=True)
        
        # Connect to SQLite database
        with sqlite3.connect(db_path) as conn:
            # Save data to the database (append mode)
            stock_data.to_sql('stocks', conn, if_exists='append', index=False)
        
        print(f"Data for {company_name} ({ticker}) saved to SQLite database.")
    except Exception as e:
        print(f"An error occurred while fetching data for {ticker}: {e}")

## Defining Database and Stock Symbols
We define:
- The SQLite database file (`stocks_data.db`) to store the stock data.
- The companies and their corresponding stock tickers for fetching data.

In [3]:
# List of companies and their tickers
stocks = [
    {'ticker': 'XOM', 'company': 'Exxon Mobil'},
    {'ticker': 'CVX', 'company': 'Chevron'},
    {'ticker': 'COP', 'company': 'ConocoPhillips'},
    {'ticker': 'T', 'company': 'AT&T'},
    {'ticker': 'TMUS', 'company': 'T-Mobile'},
    {'ticker': 'VZ', 'company': 'Verizon'}
]

# Path to SQLite database file
db_path = 'database/stocks_data.db'

## Fetching and Storing Data for All Companies
Iterates over the predefined stock symbols, fetching data for each ticker and storing it in the database. A verification step checks the database contents to confirm successful storage.

In [4]:
# Fetch and store data for each stock
for stock in stocks:
    fetch_and_store_data(stock['ticker'], stock['company'], db_path)

[*********************100%***********************]  1 of 1 completed


Data for Exxon Mobil (XOM) saved to SQLite database.


[*********************100%***********************]  1 of 1 completed


Data for Chevron (CVX) saved to SQLite database.


[*********************100%***********************]  1 of 1 completed


...

Data for AT&T (T) saved to SQLite database.


[*********************100%***********************]  1 of 1 completed


Data for T-Mobile (TMUS) saved to SQLite database.


[*********************100%***********************]  1 of 1 completed

Data for Verizon (VZ) saved to SQLite database.


## Verifying Data Storage
Finally, a query is run against the database to ensure all records are stored correctly. This provides an overview of the companies and tickers successfully saved in the database.


In [5]:
# Verify data in the SQLite database
print("\nVerifying stored data:")
with sqlite3.connect(db_path) as conn:
    query = "SELECT DISTINCT Ticker, Company FROM stocks"
    stored_data = pd.read_sql(query, conn)
    print(stored_data)


Verifying stored data:
  Ticker         Company
0    XOM     Exxon Mobil
1    CVX         Chevron
2    COP  ConocoPhillips
3      T            AT&T
4   TMUS        T-Mobile
5     VZ         Verizon
